In [207]:
import polars as pl
import boto3
import xmltodict
import duckdb
import gzip
import json 
import copy
import pandas as pd

from sqlalchemy import create_engine
from sqlalchemy import text
from sqlalchemy import Table

import config

In [176]:




s3 = boto3.client('s3',                         
                  endpoint_url='http://localhost:9000',
                  aws_access_key_id='sOAg8CTUm62YCkj5mZLt',
                  aws_secret_access_key='h0Qm2r0gz7zc0Cp3gx4gNaKVndmkMvaBlZli7DMz'
)


def list_files(client, bucket: str):
    """List files in specific S3 URL"""
    response = client.list_objects(Bucket=bucket)
    for content in response.get('Contents', []):
        yield content.get('Key')

def parsing_infos_coll(dict_from_xml: dict):
    infos_dict = dict()
    dict_entete_doc = dict_from_xml["DocumentBudgetaire"]["EnTeteDocBudgetaire"]
    infos_dict["siret_coll"] = dict_entete_doc["IdColl"]["@V"]
    infos_dict["libelle_collectivite"] = dict_entete_doc["LibelleColl"]["@V"]
    infos_dict["nature_collectivite"] = dict_entete_doc["NatCEPL"]["@V"]
    infos_dict["departement"] = dict_entete_doc.get("Departement", {}).get("@V", None)

    return infos_dict

def parsing_infos_etablissement(dict_from_xml: dict, id_doc_budg):
    infos_dict = dict()
    dict_entete_budget = dict_from_xml["DocumentBudgetaire"]["Budget"]["EnTeteBudget"]
    dict_bloc_budget = dict_from_xml["DocumentBudgetaire"]["Budget"]["BlocBudget"]

    infos_dict["id_doc_budg"] = id_doc_budg
    
    #Parsing entête budgétaire
    infos_dict["siret_etablissement"] = dict_entete_budget["IdEtab"]["@V"]
    infos_dict["libelle"] = dict_entete_budget["LibelleEtab"]["@V"]
    infos_dict["code_insee"] = dict_entete_budget.get(
        "CodInseeColl", {}).get("@V", None)
    infos_dict["nomenclature"] = dict_entete_budget["Nomenclature"]["@V"]
    
    # Parsing bloc budget
    infos_dict["exercice"] = int(dict_bloc_budget["Exer"]["@V"])
    infos_dict["nature_dec"] = dict_bloc_budget["NatDec"]["@V"]
    infos_dict["num_dec"] = int(dict_bloc_budget.get(
        "NumDec", {}).get("@V", None) or 0)
    infos_dict["nature_vote"] = dict_bloc_budget["NatFonc"]["@V"]
    infos_dict["type_budget"] = dict_bloc_budget["CodTypBud"]["@V"]
    infos_dict["id_etabl_princ"] = dict_bloc_budget.get(
        "IdEtabPal", {}).get("@V", None)

    # Parsing des lignes de budget -> json
    infos_dict["json_budget"] = generate_dict_budget(dict_from_xml)
    # duplique certaines données et non nécessaire pour l'instant donc je le commente
    #if isinstance(dict_from_xml["DocumentBudgetaire"]["Budget"]["Annexes"], dict):
    #    infos_dict["list_annexes"] = list(dict_from_xml["DocumentBudgetaire"]["Budget"]["Annexes"].keys())
    
    infos_dict["fk_siret_collectivite"] = dict_from_xml["DocumentBudgetaire"]["EnTeteDocBudgetaire"]["IdColl"]["@V"]

    return infos_dict

def generate_dict_budget(dict_from_xml: dict) -> dict:
        budget_dict = copy.deepcopy(
            dict_from_xml["DocumentBudgetaire"]["Budget"]["LigneBudget"])
        
        if isinstance(budget_dict, dict):
            for field in config.CHAMPS_LIGNE_BUDGET:
                if field in budget_dict:
                    if "@V" in budget_dict[field]:
                        budget_dict[field] = budget_dict[field]['@V']
            # for field in ["MtSup", "CaracSup"]:
            #     if field in budget_dict:
            #         if "@V" in budget_dict[field]:
            #             budget_dict[field] = { budget_dict[field]['@Code']: budget_dict[field]['@V'] }
            return budget_dict
        for idx, row in enumerate(budget_dict):
            for field in config.CHAMPS_LIGNE_BUDGET:
                if field in row:
                    if "@V" in row[field]:
                        budget_dict[idx][field] = row[field]['@V']
            for field in ["MtSup", "CaracSup"]:
                if field in row:
                    budget_dict[idx].pop(field)
            # for field in ["MtSup", "CaracSup"]:
            # if field in row:
            #     if "@V" in row[field]:
            #         budget_dict[idx][field] = { row[field]['@Code']: row[field]['@V'] }
        #erreur s'il y a qu'une seule ligne de budget
        return budget_dict

In [4]:
config.CHAMPS_LIGNE_BUDGET

['Nature',
 'LibCpte',
 'Fonction',
 'Operation',
 'ContNat',
 'ArtSpe',
 'ContFon',
 'ContOp',
 'CodRD',
 'MtBudgPrec',
 'MtRARPrec',
 'MtPropNouv',
 'MtPrev',
 'CredOuv',
 'MtReal',
 'MtRAR3112',
 'OpBudg',
 'TypOpBudg',
 'OpeCpteTiers']

In [5]:
bucket = 'data'

for file in list_files(s3, bucket):
    print(file.split("-")[1].split(".")[0])

1810875
1810878
1810879
1810884
1810887
1810890
1810893
1810896
1810899
1810902
1810905
1810908
1810911
1810914
1810915
1810918
1810923
1810926
1810929
1810932
1810935
1810938
1810941
1810944
1810947
1810950
1810953
1810956
1810957
1810962
1810965
1810968
1810971
1810972
1810973
1810976
1810979
1810984
1810989
1810992
1810995
1810998
1811001
1811004
1811007
1811008
1811013
1811014
1811019
1811022
1811025
1811028
1811031
1811034
1811037
1811040
1811043
1811046
1811049
1811052
1811053
1811058
1811061
1811064
1811065
1811066
1811067
1811076
1811079
1811082
1811085
1811088
1811091
1811094
1811097
1811098
1811099
1811100
1811103
1811110
1811115
1811118
1811121
1811122
1811125
1811130
1811133
1811136
1811139
1811142
1811143
1811146
1811147
1811152
1811155
1811156
1811161
1811164
1811169
1811172
1811175
1811178
1811181
1811184
1811187
1811190
1811193
1811196
1811199
1811202
1811205
1811208
1811211
1811214
1811215
1811220
1811223
1811226
1811229
1811230
1811235
1811238
1811241
1811244
1811245


In [6]:
bucket = 'data'
with pl.StringCache():
    coll_df = pl.DataFrame()
    for file in list_files(s3, bucket):

        response = s3.get_object(Bucket=bucket,
                            Key=file)
        dict_from_xml = xmltodict.parse(
                gzip.GzipFile(fileobj=response["Body"]), dict_constructor=dict
            )
        temp_df = pl.DataFrame(
                parsing_infos_coll(dict_from_xml),
                schema={
                    "siret_coll": pl.UInt32,
                    "libelle_collectivite": pl.Utf8,
                    "nature_collectivite": pl.Categorical,
                    "departement": pl.Categorical,
                },
            )
        coll_df = pl.concat([coll_df, temp_df], how="vertical")

In [7]:
coll_df

siret_coll,libelle_collectivite,nature_collectivite,departement
str,str,cat,cat
"""20009162700022…","""REGIE SPANC""","""Communes""","""12"""
"""20009162700014…","""SYNDICAT MIXTE…","""Commune""","""12"""
"""20009162700022…","""REGIE SPANC""","""Communes""","""12"""
"""20008531400017…","""MAIRIE SAINT-D…","""Commune""",null
"""26240002100016…","""AGONAC""","""Commune""","""24"""
"""21240002200019…","""AGONAC""","""Commune""","""24"""
"""26800519600012…","""CCAS DE LIERCO…","""Commune""","""80"""
"""21240284600019…","""MONTAGNAC D'AU…","""Commune""","""24"""
"""26800519600012…","""CCAS DE LIERCO…","""Commune""","""80"""


In [8]:
path = './collectivites.parquet' # already created on S3
coll_df.write_parquet(path)


In [9]:
with pl.StringCache():
    coll_df = pl.DataFrame(schema=["id_doc_budg",
                    "siret_etablissement",
                    "libelle",
                    "code_insee",
                    "nomenclature",
                    "exercice",
                    "nature_dec",
                    "num_dec",
                    "nature_vote",
                    "type_budget",
                    "id_etabl_princ",
                    "json_budget",
                    "fk_siret_collectivite","Nature","LibCpte","Fonction","Operation","ContNat","ArtSpe","ContFon",
                 "ContOp","CodRD","MtBudgPrec","MtRARPrec","MtPropNouv","MtPrev","CredOuv",
                 "MtReal","MtRAR3112","OpBudg","TypOpBudg","OpeCpteTiers"],
                 schema_overrides={"id_doc_budg": pl.Utf8},
                 infer_schema_length=0
                 )
    con.begin()
    count = 0
    for file in list_files(s3, bucket):
            id_doc_budg = file.split("-")[1].split(".")[0]

            response = s3.get_object(Bucket=bucket,
                                Key=file)
            dict_from_xml = xmltodict.parse(
                    gzip.GzipFile(fileobj=response["Body"]), dict_constructor=dict
                )
            temp_df = pl.from_dicts(
                    parsing_infos_etablissement(dict_from_xml, id_doc_budg),
                schema_overrides={"json_budget": pl.Struct, "id_doc_budg": pl.UInt32},
                infer_schema_length=0
                ).unnest("json_budget")
            print(len(temp_df.columns))
            #coll_df = pl.concat([coll_df, temp_df], how="vertical")
            #coll_df.select(pl.all().append(temp_df))
            #coll_df.extend(temp_df)
            #coll_df = coll_df.vstack(temp_df)
            con.append("test_df_table", temp_df)
    con.commit()
path = './doc_budg.parquet' # already created on S3
coll_df.write_parquet(path)

NameError: name 'con' is not defined

In [21]:
con = duckdb.connect('my-db.duckdb')
con.execute('CREATE OR REPLACE TABLE ab_budg AS SELECT * FROM temp_df')
con.sql('select * from ab_budg')

┌────────┬─────────┬──────────┬───────────┬───┬─────────────┬────────────────┬─────────────┬──────────────────────┐
│ Nature │ LibCpte │ Fonction │ Operation │ … │ type_budget │ id_etabl_princ │ json_budget │ fk_siret_collectiv…  │
│ int64  │  int64  │  int64   │   int64   │   │   varchar   │     int32      │    int64    │       varchar        │
├────────┼─────────┼──────────┼───────────┼───┼─────────────┼────────────────┼─────────────┼──────────────────────┤
│   NULL │    NULL │     NULL │      NULL │ … │ P           │           NULL │        NULL │ 20009162700022       │
│   NULL │    NULL │     NULL │      NULL │ … │ P           │           NULL │        NULL │ 20009162700022       │
│   NULL │    NULL │     NULL │      NULL │ … │ P           │           NULL │        NULL │ 20009162700022       │
│   NULL │    NULL │     NULL │      NULL │ … │ P           │           NULL │        NULL │ 20009162700022       │
│   NULL │    NULL │     NULL │      NULL │ … │ P           │           

In [ ]:
coll_df

id_doc_budg,siret_etablissement,libelle,code_insee,nomenclature,exercice,nature_dec,num_dec,nature_vote,type_budget,id_etabl_princ,json_budget,fk_siret_collectivite,Nature,LibCpte,Fonction,Operation,ContNat,ArtSpe,ContFon,ContOp,CodRD,MtBudgPrec,MtRARPrec,MtPropNouv,MtPrev,CredOuv,MtReal,MtRAR3112,OpBudg,TypOpBudg,OpeCpteTiers,MtSup,CaracSup
f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32


In [ ]:
df = pd.read_parquet('./doc_budg.parquet' )
df.head(5)

,id_doc_budg,siret_etablissement,libelle,code_insee,nomenclature,exercice,nature_dec,num_dec,nature_vote,type_budget,...,MtBudgPrec,MtRARPrec,MtPropNouv,MtPrev,CredOuv,MtReal,MtRAR3112,OpBudg,TypOpBudg,OpeCpteTiers


In [195]:
champs_budg = config.CHAMPS_LIGNE_BUDGET

def _all_annexe_columns(df, fields):
    all_columns = copy.deepcopy(fields)
    columns = list(df.columns)
    for i in all_columns:
        columns.append(i)
    return columns

def explode_annexe_json_into_rows_first_way(df, fields):
    all_columns = _all_annexe_columns(df, fields)
    df['json_budget'] = df.json_budget.apply(lambda x:eval(str(x)))
    temp = df.groupby('id_doc_budg').json_budget.apply(lambda x: pd.DataFrame(x.values[0], index=[0]).reset_index())
    df.drop(columns="json_budget", inplace=True)
    df_result = df.merge(temp, left_on='id_doc_budg', right_on='id_doc_budg')
    return df_result

def explode_annexe_json_into_rows_second_way(df, fields):
    all_columns = _all_annexe_columns(df, fields)
    s= df.set_index('id_doc_budg').json_budget.apply(lambda x:eval(str(x))).explode()
    temp = pd.DataFrame(s.tolist(), index = s.index).reset_index()
    df.drop(columns="json_budget", inplace=True)
    df_result = temp.merge(df, left_on='id_doc_budg', right_on='id_doc_budg')
    return df_result.reindex(columns = all_columns)

In [206]:

engine = create_engine('sqlite://', echo=True)


result = pd.DataFrame()
count = 0
for file in list_files(s3, bucket):
    print(file)
    id_doc_budg = file.split("-")[1].split(".")[0]

    response = s3.get_object(Bucket=bucket,
                        Key=file)
    dict_from_xml = xmltodict.parse(
            gzip.GzipFile(fileobj=response["Body"]), dict_constructor=dict
        )
    temp_df = pd.DataFrame.from_dict(
            parsing_infos_etablissement(dict_from_xml, id_doc_budg),
        )
    temp_df = explode_annexe_json_into_rows_first_way(temp_df, config.CHAMPS_LIGNE_BUDGET)
    
    if count == 0:
        temp_df.to_sql('ab_budg', con=engine)
        count = 1
    else:
        temp_df.to_sql('ab_budg', con=engine, if_exists='append')


    
    # result = pd.concat([result, temp_df], axis=1, ignore_index=True, sort=False)
    
    #con.execute('INSERT INTO ab_budg SELECT * FROM temp_df')
    



20200801-1810875.xml.gz
2023-07-06 13:28:22,011 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-07-06 13:28:22,014 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("ab_budg")
2023-07-06 13:28:22,015 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-07-06 13:28:22,016 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("ab_budg")
2023-07-06 13:28:22,016 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-07-06 13:28:22,019 INFO sqlalchemy.engine.Engine 
CREATE TABLE ab_budg (
	level_0 BIGINT, 
	id_doc_budg TEXT, 
	siret_etablissement TEXT, 
	libelle TEXT, 
	code_insee TEXT, 
	nomenclature TEXT, 
	exercice BIGINT, 
	nature_dec TEXT, 
	num_dec BIGINT, 
	nature_vote TEXT, 
	type_budget TEXT, 
	id_etabl_princ TEXT, 
	fk_siret_collectivite TEXT, 
	"index" BIGINT, 
	"Nature" TEXT, 
	"LibCpte" TEXT, 
	"ContNat" TEXT, 
	"ArtSpe" TEXT, 
	"CodRD" TEXT, 
	"MtBudgPrec" TEXT, 
	"MtRARPrec" TEXT, 
	"MtPropNouv" TEXT, 
	"MtPrev" TEXT, 
	"CredOuv" TEXT, 
	"MtReal" TEXT, 
	"MtRAR3112" TEXT, 
	"OpBudg

OperationalError: (sqlite3.OperationalError) table ab_budg has no column named TypOpBudg
[SQL: INSERT INTO ab_budg (level_0, id_doc_budg, siret_etablissement, libelle, code_insee, nomenclature, exercice, nature_dec, num_dec, nature_vote, type_budget, id_etabl_princ, fk_siret_collectivite, "index", "Nature", "LibCpte", "ContNat", "ArtSpe", "CodRD", "MtBudgPrec", "MtRARPrec", "MtPropNouv", "MtPrev", "OpBudg", "TypOpBudg") VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)]
[parameters: [(0, '1810884', '20008531400017', 'MAIRIE SAINT-DIZIER-MASBARAUD Budget principal', None, 'M14-M14_COM_500_3500', 2020, '01', 0, '1', 'P', None, '20008531400017', 0, '023', "Virement à la section d'investissement", '023', 'false', 'D', '0', '0', '238400.35', '238400.35', '1', '2'), (1, '1810884', '20008531400017', 'MAIRIE SAINT-DIZIER-MASBARAUD Budget principal', None, 'M14-M14_COM_500_3500', 2020, '01', 0, '1', 'P', None, '20008531400017', 0, '023', "Virement à la section d

In [203]:


with engine.connect() as conn:

   result = conn.execute(text("SELECT * FROM ab_budg")).fetchall()

2023-07-06 13:26:21,147 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-07-06 13:26:21,151 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("ab_budg")
2023-07-06 13:26:21,152 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-07-06 13:26:21,153 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("ab_budg")
2023-07-06 13:26:21,154 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-07-06 13:26:21,176 INFO sqlalchemy.engine.Engine 
CREATE TABLE ab_budg (
	level_0 BIGINT, 
	id_doc_budg TEXT, 
	siret_etablissement TEXT, 
	libelle TEXT, 
	code_insee TEXT, 
	nomenclature TEXT, 
	exercice BIGINT, 
	nature_dec TEXT, 
	num_dec BIGINT, 
	nature_vote TEXT, 
	type_budget TEXT, 
	id_etabl_princ TEXT, 
	fk_siret_collectivite TEXT, 
	"index" BIGINT, 
	"Nature" TEXT, 
	"Fonction" TEXT, 
	"ContNat" TEXT, 
	"ArtSpe" TEXT, 
	"CodRD" TEXT, 
	"MtBudgPrec" TEXT, 
	"MtPropNouv" TEXT, 
	"MtPrev" TEXT, 
	"CredOuv" TEXT, 
	"MtReal" TEXT, 
	"OpBudg" TEXT
)


2023-07-06 13:26:21,179 INFO sqlalchemy.engine.Engin

In [204]:
result

[(0, '1811268', '21260235300014', 'BUDGET PRINCIPAL', '26235', 'M14-M14_COM_SUP3500', 2020, '01', 0, '3', 'P', '21260235300014', '21260235300014', 0, '60611', '211', '011', 'false', 'D', '6050.00', '6770.00', '6770.00', '6050.00', '4682.03', '0'),
 (1, '1811268', '21260235300014', 'BUDGET PRINCIPAL', '26235', 'M14-M14_COM_SUP3500', 2020, '01', 0, '3', 'P', '21260235300014', '21260235300014', 0, '60611', '211', '011', 'false', 'D', '6050.00', '6770.00', '6770.00', '6050.00', '4682.03', '0'),
 (2, '1811268', '21260235300014', 'BUDGET PRINCIPAL', '26235', 'M14-M14_COM_SUP3500', 2020, '01', 0, '3', 'P', '21260235300014', '21260235300014', 0, '60611', '211', '011', 'false', 'D', '6050.00', '6770.00', '6770.00', '6050.00', '4682.03', '0'),
 (3, '1811268', '21260235300014', 'BUDGET PRINCIPAL', '26235', 'M14-M14_COM_SUP3500', 2020, '01', 0, '3', 'P', '21260235300014', '21260235300014', 0, '60611', '211', '011', 'false', 'D', '6050.00', '6770.00', '6770.00', '6050.00', '4682.03', '0'),
 (4, '18

In [186]:
con = duckdb.connect('my-db.duckdb')
con.execute('CREATE OR REPLACE TABLE ab_budg AS SELECT * FROM temp_df')
con.sql('select * from ab_budg')
#con.append("ab_budg", temp_df)
temp_df.dtypes

id_doc_budg               object
siret_etablissement       object
libelle                   object
code_insee                object
nomenclature              object
exercice                   int64
nature_dec                object
num_dec                    int64
nature_vote               object
type_budget               object
id_etabl_princ            object
json_budget              float64
fk_siret_collectivite     object
Nature                    object
LibCpte                  float64
Fonction                  object
Operation                float64
ContNat                   object
ArtSpe                    object
ContFon                  float64
ContOp                   float64
CodRD                     object
MtBudgPrec                object
MtRARPrec                float64
MtPropNouv                object
MtPrev                    object
CredOuv                   object
MtReal                    object
MtRAR3112                float64
OpBudg                    object
TypOpBudg 

In [ ]:
coll_df

id_doc_budg,siret_etablissement,libelle,code_insee,nomenclature,exercice,nature_dec,num_dec,nature_vote,type_budget,id_etabl_princ,json_budget,fk_siret_collectivite,Nature,LibCpte,Fonction,Operation,ContNat,ArtSpe,ContFon,ContOp,CodRD,MtBudgPrec,MtRARPrec,MtPropNouv,MtPrev,CredOuv,MtReal,MtRAR3112,OpBudg,TypOpBudg,OpeCpteTiers
str,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32


In [177]:
file = list(list_files(s3, bucket))[5]

print(file)
id_doc_budg = file.split("-")[1].split(".")[0]

response = s3.get_object(Bucket=bucket,
                    Key=file)
dict_from_xml = xmltodict.parse(
        gzip.GzipFile(fileobj=response["Body"]), dict_constructor=dict
    )
temp_df = pl.from_dicts(
        parsing_infos_etablissement(dict_from_xml, id_doc_budg),
    schema_overrides={"json_budget": pl.Struct},
    infer_schema_length=None
    ).unnest("json_budget")

duckdb.sql('SELECT * FROM temp_df').show()

20200801-1810890.xml.gz
┌─────────────┬─────────────────────┬──────────────────┬───┬─────────┬───────────┬───────────┬──────────────────────┐
│ id_doc_budg │ siret_etablissement │     libelle      │ … │ OpBudg  │ TypOpBudg │ MtRARPrec │ fk_siret_collectiv…  │
│   varchar   │       varchar       │     varchar      │   │ varchar │  varchar  │  varchar  │       varchar        │
├─────────────┼─────────────────────┼──────────────────┼───┼─────────┼───────────┼───────────┼──────────────────────┤
│ 1810890     │ 21240002200019      │ COMMUNE D'AGONAC │ … │ 0       │ NULL      │ NULL      │ 21240002200019       │
│ 1810890     │ 21240002200019      │ COMMUNE D'AGONAC │ … │ 0       │ NULL      │ NULL      │ 21240002200019       │
│ 1810890     │ 21240002200019      │ COMMUNE D'AGONAC │ … │ 0       │ NULL      │ NULL      │ 21240002200019       │
│ 1810890     │ 21240002200019      │ COMMUNE D'AGONAC │ … │ 0       │ NULL      │ NULL      │ 21240002200019       │
│ 1810890     │ 21240002200019  

In [191]:

count = 0
file = "20200801-1810947.xml.gz"
    
print(file)
id_doc_budg = file.split("-")[1].split(".")[0]

response = s3.get_object(Bucket=bucket,
                    Key=file)
dict_from_xml = xmltodict.parse(
        gzip.GzipFile(fileobj=response["Body"]), dict_constructor=dict
    )
temp_df = pd.DataFrame.from_dict(
        parsing_infos_etablissement(dict_from_xml, id_doc_budg),
    )
    
all_columns = _all_annexe_columns(temp_df, config.CHAMPS_LIGNE_BUDGET)
temp_df['json_budget'] = temp_df.json_budget.apply(lambda x:eval(str(x)))
temp = temp_df.groupby('id_doc_budg').json_budget.apply(lambda x: pd.DataFrame(x.values[0], index=[0]).reset_index())
temp_df.drop(columns="json_budget", inplace=True)
df_result = temp_df.merge(temp, left_on='id_doc_budg', right_on='id_doc_budg')
df_result.reindex(columns = all_columns)


20200801-1810947.xml.gz


In [192]:
df_result

,id_doc_budg,siret_etablissement,libelle,code_insee,nomenclature,exercice,nature_dec,num_dec,nature_vote,type_budget,...,Nature,LibCpte,Operation,ArtSpe,ContOp,CodRD,MtPropNouv,MtPrev,CredOuv,OpBudg
0,1810947,21480056700072,EAU ASSAINISSEMENT,48056,M4-M49_D,2020,01,0,1,A,...,2315,Assainissement La Rocherousse,116,false,116,D,70000.00,70000.00,70000.00,0
1,1810947,21480056700072,EAU ASSAINISSEMENT,48056,M4-M49_D,2020,01,0,1,A,...,2315,Assainissement La Rocherousse,116,false,116,D,70000.00,70000.00,70000.00,0
2,1810947,21480056700072,EAU ASSAINISSEMENT,48056,M4-M49_D,2020,01,0,1,A,...,2315,Assainissement La Rocherousse,116,false,116,D,70000.00,70000.00,70000.00,0
3,1810947,21480056700072,EAU ASSAINISSEMENT,48056,M4-M49_D,2020,01,0,1,A,...,2315,Assainissement La Rocherousse,116,false,116,D,70000.00,70000.00,70000.00,0
4,1810947,21480056700072,EAU ASSAINISSEMENT,48056,M4-M49_D,2020,01,0,1,A,...,2315,Assainissement La Rocherousse,116,false,116,D,70000.00,70000.00,70000.00,0
5,1810947,21480056700072,EAU ASSAINISSEMENT,48056,M4-M49_D,2020,01,0,1,A,...,2315,Assainissement La Rocherousse,116,false,116,D,70000.00,70000.00,70000.00,0
6,1810947,21480056700072,EAU ASSAINISSEMENT,48056,M4-M49_D,2020,01,0,1,A,...,2315,Assainissement La Rocherousse,116,false,116,D,70000.00,70000.00,70000.00,0
7,1810947,21480056700072,EAU ASSAINISSEMENT,48056,M4-M49_D,2020,01,0,1,A,...,2315,Assainissement La Rocherousse,116,false,116,D,70000.00,70000.00,70000.00,0
8,1810947,21480056700072,EAU ASSAINISSEMENT,48056,M4-M49_D,2020,01,0,1,A,...,2315,Assainissement La Rocherousse,116,false,116,D,70000.00,70000.00,70000.00,0
9,1810947,21480056700072,EAU ASSAINISSEMENT,48056,M4-M49_D,2020,01,0,1,A,...,2315,Assainissement La Rocherousse,116,false,116,D,70000.00,70000.00,70000.00,0
